<h1 style="font-family: 'Times New Roman'; letter-spacing: 0.08em;">Importing Libraries</h1>

In [1]:
import pandas as pd
import ast
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

<h1 style="font-family: 'Times New Roman'; letter-spacing: 0.08em;">Diaplaying Datasets</h1>

In [2]:
df1 = pd.read_csv('dishes.csv')
df1.head(2)

,Unnamed: 0,country,description,id_,keywords,is_shoppable,language,name,slug,video_url,...,total_time,ratings_negative,ratings_positive,score,protein,fat,calories,sugar,carbohydrates,fiber
0,0,US,"Nothing says Sunday morning like a warm, gooey...",868,"bake sale, breakfast, cinnamon buns, cinnamon ...",True,eng,Homemade Cinnamon Rolls,homemade-cinnamon-rolls,https://vid.tasty.co/output/25812/mp4_640x640/...,...,151,475,17340,98.0,7.0,21.0,479.0,24.0,63.0,1.0
1,1,US,NaN,6124,"1000 whisk omelette, 3 ingredient mac & cheese...",True,eng,Whipped Coffee,whipped-coffee,https://vid.tasty.co/output/163616/landscape_4...,...,15,777,8483,92.0,0.0,0.0,69.0,18.0,18.0,0.0


In [3]:
df2= pd.read_json('ingredient_and_instructions.json').T
df2.head()

,ingredient_sections,instructions
homemade-cinnamon-rolls,"[{'name': 'Dough', 'ingredients': [{'extra_com...",[{'display_text': 'Generously butter two dispo...
whipped-coffee,"[{'name': '', 'ingredients': [{'extra_comment'...","[{'display_text': 'Add the hot water, sugar, a..."
fluffy-perfect-pancakes,"[{'name': '', 'ingredients': [{'extra_comment'...",[{'display_text': 'Whisk together the flour an...
tasty-101-cinnamon-rolls,"[{'name': 'Dough', 'ingredients': [{'extra_com...",[{'display_text': 'Make the dough: In a large ...
healthy-banana-pancakes,"[{'name': '', 'ingredients': [{'extra_comment'...",[{'display_text': 'Mash bananas in a large bow...


<h1 style="font-family: 'Times New Roman'; letter-spacing: 0.08em;">Cleaning The Data</h1>

<h2 style="font-family: 'Times New Roman'; letter-spacing: 0.08em;">Unneccessary Columns</h2>

In [4]:
df1 = df1.drop(['Unnamed: 0', 'is_licensed_video', 'is_shoppable', 'description', 'country', 'language',
                'is_community', 'inspired_by', 'linked_recipes', 'cook_time', 'prep_time', 'ratings_negative',
                'keywords', 'tags', 'id_', 'total_time', 'slug'], axis=1)
df1.head(2)

,name,video_url,thumbnail_url,ratings_positive,score,protein,fat,calories,sugar,carbohydrates,fiber
0,Homemade Cinnamon Rolls,https://vid.tasty.co/output/25812/mp4_640x640/...,https://img.buzzfeed.com/video-api-prod/assets...,17340,98.0,7.0,21.0,479.0,24.0,63.0,1.0
1,Whipped Coffee,https://vid.tasty.co/output/163616/landscape_4...,https://img.buzzfeed.com/video-api-prod/assets...,8483,92.0,0.0,0.0,69.0,18.0,18.0,0.0


In [5]:
df2.head(2)

,ingredient_sections,instructions
homemade-cinnamon-rolls,"[{'name': 'Dough', 'ingredients': [{'extra_com...",[{'display_text': 'Generously butter two dispo...
whipped-coffee,"[{'name': '', 'ingredients': [{'extra_comment'...","[{'display_text': 'Add the hot water, sugar, a..."


In [6]:
df= pd.concat([df1, df2.reset_index(drop=True)], axis=1, join='outer')
df.head(2)

,name,video_url,thumbnail_url,ratings_positive,score,protein,fat,calories,sugar,carbohydrates,fiber,ingredient_sections,instructions
0,Homemade Cinnamon Rolls,https://vid.tasty.co/output/25812/mp4_640x640/...,https://img.buzzfeed.com/video-api-prod/assets...,17340,98.0,7.0,21.0,479.0,24.0,63.0,1.0,"[{'name': 'Dough', 'ingredients': [{'extra_com...",[{'display_text': 'Generously butter two dispo...
1,Whipped Coffee,https://vid.tasty.co/output/163616/landscape_4...,https://img.buzzfeed.com/video-api-prod/assets...,8483,92.0,0.0,0.0,69.0,18.0,18.0,0.0,"[{'name': '', 'ingredients': [{'extra_comment'...","[{'display_text': 'Add the hot water, sugar, a..."


<h2 style="font-family: 'Times New Roman'; letter-spacing: 0.08em;">Null Values</h2>

In [7]:
null_df = df.isnull().sum()[df.isnull().sum() > 0].sort_values().to_frame().rename(columns={0: "Total Missing values"})
null_df["% of Missing Values"] = round(null_df["Total Missing values"]/len(df)*100, 2)
null_df["Feature Data Type"] = df[null_df.index.tolist()].dtypes

null_df

,Total Missing values,% of Missing Values,Feature Data Type
score,75,1.51,float64
protein,499,10.06,float64
fat,499,10.06,float64
calories,499,10.06,float64
carbohydrates,499,10.06,float64
sugar,500,10.08,float64
fiber,500,10.08,float64
video_url,530,10.68,object


In [8]:
print(df.shape)

(4961, 13)


In [9]:
df = df.dropna(subset=['video_url'])
print(df.shape)

(4431, 13)


<h2 style="font-family: 'Times New Roman'; letter-spacing: 0.08em;">Missing Values</h2>

In [10]:
null_df = df.isnull().sum()[df.isnull().sum() > 0].sort_values().to_frame().rename(columns={0: "Total Missing values"})
null_df["% of Missing Values"] = round(null_df["Total Missing values"]/len(df)*100, 2)
null_df["Feature Data Type"] = df[null_df.index.tolist()].dtypes

null_df

,Total Missing values,% of Missing Values,Feature Data Type
score,16,0.36,float64
protein,428,9.66,float64
fat,428,9.66,float64
calories,428,9.66,float64
carbohydrates,428,9.66,float64
sugar,429,9.68,float64
fiber,429,9.68,float64


<h3 style="font-family: 'Times New Roman'; letter-spacing: 0.08em;">Filling Numeric Data With Mean</h3>

In [11]:
df['carbohydrates'] = df['carbohydrates'].fillna(df['carbohydrates'].mean())
df['calories'] = df['calories'].fillna(df['calories'].mean())
df['protein'] = df['protein'].fillna(df['protein'].mean())
df['sugar'] = df['sugar'].fillna(df['sugar'].mean())
df['fiber'] = df['fiber'].fillna(df['fiber'].mean())	
df['score'] = df['score'].fillna(df['score'].mean())
df['fat'] = df['fat'].fillna(df['fat'].mean())

In [12]:
null_df = df.isnull().sum()[df.isnull().sum() > 0].sort_values().to_frame().rename(columns={0: "Total Missing values"})
null_df["% of Missing Values"] = round(null_df["Total Missing values"]/len(df)*100, 2)
null_df["Feature Data Type"] = df[null_df.index.tolist()].dtypes

null_df

,Total Missing values,% of Missing Values,Feature Data Type


In [13]:
print(df.shape)

(4431, 13)


In [14]:
df.head(2)

,name,video_url,thumbnail_url,ratings_positive,score,protein,fat,calories,sugar,carbohydrates,fiber,ingredient_sections,instructions
0,Homemade Cinnamon Rolls,https://vid.tasty.co/output/25812/mp4_640x640/...,https://img.buzzfeed.com/video-api-prod/assets...,17340,98.0,7.0,21.0,479.0,24.0,63.0,1.0,"[{'name': 'Dough', 'ingredients': [{'extra_com...",[{'display_text': 'Generously butter two dispo...
1,Whipped Coffee,https://vid.tasty.co/output/163616/landscape_4...,https://img.buzzfeed.com/video-api-prod/assets...,8483,92.0,0.0,0.0,69.0,18.0,18.0,0.0,"[{'name': '', 'ingredients': [{'extra_comment'...","[{'display_text': 'Add the hot water, sugar, a..."


In [15]:
df.columns

Index(['name', 'video_url', 'thumbnail_url', 'ratings_positive', 'score',
       'protein', 'fat', 'calories', 'sugar', 'carbohydrates', 'fiber',
       'ingredient_sections', 'instructions'],
      dtype='object')

In [16]:
df.iloc[0]['ingredient_sections']

[{'name': 'Dough',
  'ingredients': [{'extra_comment': 'melted',
    'name': 'unsalted butter',
    'primary_unit': {'quantity': '½', 'display': 'cup'},
    'metric_unit': {'quantity': '115', 'display': 'g'}},
   {'extra_comment': 'warm to the touch',
    'name': 'whole milk',
    'primary_unit': {'quantity': '2', 'display': 'cups'},
    'metric_unit': {'quantity': '480', 'display': 'mL'}},
   {'extra_comment': '',
    'name': 'granulated sugar',
    'primary_unit': {'quantity': '½', 'display': 'cup'},
    'metric_unit': {'quantity': '100', 'display': 'g'}},
   {'extra_comment': '',
    'name': 'active dry yeast',
    'primary_unit': {'quantity': '2 ¼', 'display': 'teaspoons'},
    'metric_unit': None},
   {'extra_comment': 'divided',
    'name': 'flour',
    'primary_unit': {'quantity': '5', 'display': 'cups'},
    'metric_unit': {'quantity': '625', 'display': 'g'}},
   {'extra_comment': '',
    'name': 'baking powder',
    'primary_unit': {'quantity': '1', 'display': 'teaspoon'},
   

In [17]:
def extract_names(data):
  names = []
  for section in data:
    if isinstance(section, dict):  
      if "name" in section and section["name"]:
        names.append(section["name"])
      if "ingredients" in section:
        if isinstance(section["ingredients"], list):
          names.extend([ingredient["name"]
                      for ingredient in section["ingredients"] if ingredient["name"]])
  return names


def clean_ingredients(df):
  df["cleaned_ingredients"] = df["ingredient_sections"].apply(extract_names)
  return df

df = clean_ingredients(df.copy())
df.head()

,name,video_url,thumbnail_url,ratings_positive,score,protein,fat,calories,sugar,carbohydrates,fiber,ingredient_sections,instructions,cleaned_ingredients
0,Homemade Cinnamon Rolls,https://vid.tasty.co/output/25812/mp4_640x640/...,https://img.buzzfeed.com/video-api-prod/assets...,17340,98.0,7.0,21.0,479.0,24.0,63.0,1.0,"[{'name': 'Dough', 'ingredients': [{'extra_com...",[{'display_text': 'Generously butter two dispo...,"[Dough, unsalted butter, whole milk, granulate..."
1,Whipped Coffee,https://vid.tasty.co/output/163616/landscape_4...,https://img.buzzfeed.com/video-api-prod/assets...,8483,92.0,0.0,0.0,69.0,18.0,18.0,0.0,"[{'name': '', 'ingredients': [{'extra_comment'...","[{'display_text': 'Add the hot water, sugar, a...","[hot water, sugar, instant coffee powder, milk..."
2,Fluffy Pancakes,https://vid.tasty.co/output/65088/landscape_48...,https://img.buzzfeed.com/thumbnailer-prod-us-e...,5009,83.0,36.0,50.0,1102.0,12.0,123.0,3.0,"[{'name': '', 'ingredients': [{'extra_comment'...",[{'display_text': 'Whisk together the flour an...,"[flour, baking powder, milk, butter, egg yolks..."
3,How to Make Cinnamon Rolls,https://vid.tasty.co/output/121022/landscape_4...,https://img.buzzfeed.com/thumbnailer-prod-us-e...,6649,97.0,8.0,25.0,562.0,28.0,74.0,1.0,"[{'name': 'Dough', 'ingredients': [{'extra_com...",[{'display_text': 'Make the dough: In a large ...,"[Dough, whole milk, sugar, unsalted butter, ac..."
4,Healthy Banana Pancakes,https://vid.tasty.co/output/27116/mp4_640x640/...,https://img.buzzfeed.com/thumbnailer-prod-us-e...,3853,87.0,7.0,4.0,184.0,9.0,30.0,4.0,"[{'name': '', 'ingredients': [{'extra_comment'...",[{'display_text': 'Mash bananas in a large bow...,"[ripe bananas, eggs, vanilla extract, quick-co..."


In [18]:
df.head(2)

,name,video_url,thumbnail_url,ratings_positive,score,protein,fat,calories,sugar,carbohydrates,fiber,ingredient_sections,instructions,cleaned_ingredients
0,Homemade Cinnamon Rolls,https://vid.tasty.co/output/25812/mp4_640x640/...,https://img.buzzfeed.com/video-api-prod/assets...,17340,98.0,7.0,21.0,479.0,24.0,63.0,1.0,"[{'name': 'Dough', 'ingredients': [{'extra_com...",[{'display_text': 'Generously butter two dispo...,"[Dough, unsalted butter, whole milk, granulate..."
1,Whipped Coffee,https://vid.tasty.co/output/163616/landscape_4...,https://img.buzzfeed.com/video-api-prod/assets...,8483,92.0,0.0,0.0,69.0,18.0,18.0,0.0,"[{'name': '', 'ingredients': [{'extra_comment'...","[{'display_text': 'Add the hot water, sugar, a...","[hot water, sugar, instant coffee powder, milk..."


In [19]:
print(df.shape)

(4431, 14)


In [20]:
# beacuse it is fallen video url
df = df[df['name'] != 'One-Pot Cheesy Lemon Chicken Pasta']

In [21]:
df.to_csv('Food.csv', index=False)

In [22]:
df = pd.read_csv('Food.csv')
df.head()

,name,video_url,thumbnail_url,ratings_positive,score,protein,fat,calories,sugar,carbohydrates,fiber,ingredient_sections,instructions,cleaned_ingredients
0,Homemade Cinnamon Rolls,https://vid.tasty.co/output/25812/mp4_640x640/...,https://img.buzzfeed.com/video-api-prod/assets...,17340,98.0,7.0,21.0,479.0,24.0,63.0,1.0,"[{'name': 'Dough', 'ingredients': [{'extra_com...",[{'display_text': 'Generously butter two dispo...,"['Dough', 'unsalted butter', 'whole milk', 'gr..."
1,Whipped Coffee,https://vid.tasty.co/output/163616/landscape_4...,https://img.buzzfeed.com/video-api-prod/assets...,8483,92.0,0.0,0.0,69.0,18.0,18.0,0.0,"[{'name': '', 'ingredients': [{'extra_comment'...","[{'display_text': 'Add the hot water, sugar, a...","['hot water', 'sugar', 'instant coffee powder'..."
2,Fluffy Pancakes,https://vid.tasty.co/output/65088/landscape_48...,https://img.buzzfeed.com/thumbnailer-prod-us-e...,5009,83.0,36.0,50.0,1102.0,12.0,123.0,3.0,"[{'name': '', 'ingredients': [{'extra_comment'...",[{'display_text': 'Whisk together the flour an...,"['flour', 'baking powder', 'milk', 'butter', '..."
3,How to Make Cinnamon Rolls,https://vid.tasty.co/output/121022/landscape_4...,https://img.buzzfeed.com/thumbnailer-prod-us-e...,6649,97.0,8.0,25.0,562.0,28.0,74.0,1.0,"[{'name': 'Dough', 'ingredients': [{'extra_com...",[{'display_text': 'Make the dough: In a large ...,"['Dough', 'whole milk', 'sugar', 'unsalted but..."
4,Healthy Banana Pancakes,https://vid.tasty.co/output/27116/mp4_640x640/...,https://img.buzzfeed.com/thumbnailer-prod-us-e...,3853,87.0,7.0,4.0,184.0,9.0,30.0,4.0,"[{'name': '', 'ingredients': [{'extra_comment'...",[{'display_text': 'Mash bananas in a large bow...,"['ripe bananas', 'eggs', 'vanilla extract', 'q..."


In [23]:
df.head()

,name,video_url,thumbnail_url,ratings_positive,score,protein,fat,calories,sugar,carbohydrates,fiber,ingredient_sections,instructions,cleaned_ingredients
0,Homemade Cinnamon Rolls,https://vid.tasty.co/output/25812/mp4_640x640/...,https://img.buzzfeed.com/video-api-prod/assets...,17340,98.0,7.0,21.0,479.0,24.0,63.0,1.0,"[{'name': 'Dough', 'ingredients': [{'extra_com...",[{'display_text': 'Generously butter two dispo...,"['Dough', 'unsalted butter', 'whole milk', 'gr..."
1,Whipped Coffee,https://vid.tasty.co/output/163616/landscape_4...,https://img.buzzfeed.com/video-api-prod/assets...,8483,92.0,0.0,0.0,69.0,18.0,18.0,0.0,"[{'name': '', 'ingredients': [{'extra_comment'...","[{'display_text': 'Add the hot water, sugar, a...","['hot water', 'sugar', 'instant coffee powder'..."
2,Fluffy Pancakes,https://vid.tasty.co/output/65088/landscape_48...,https://img.buzzfeed.com/thumbnailer-prod-us-e...,5009,83.0,36.0,50.0,1102.0,12.0,123.0,3.0,"[{'name': '', 'ingredients': [{'extra_comment'...",[{'display_text': 'Whisk together the flour an...,"['flour', 'baking powder', 'milk', 'butter', '..."
3,How to Make Cinnamon Rolls,https://vid.tasty.co/output/121022/landscape_4...,https://img.buzzfeed.com/thumbnailer-prod-us-e...,6649,97.0,8.0,25.0,562.0,28.0,74.0,1.0,"[{'name': 'Dough', 'ingredients': [{'extra_com...",[{'display_text': 'Make the dough: In a large ...,"['Dough', 'whole milk', 'sugar', 'unsalted but..."
4,Healthy Banana Pancakes,https://vid.tasty.co/output/27116/mp4_640x640/...,https://img.buzzfeed.com/thumbnailer-prod-us-e...,3853,87.0,7.0,4.0,184.0,9.0,30.0,4.0,"[{'name': '', 'ingredients': [{'extra_comment'...",[{'display_text': 'Mash bananas in a large bow...,"['ripe bananas', 'eggs', 'vanilla extract', 'q..."


### Preprocess Text in Instructions Column 

In [24]:
def clean_instructions(instructions):
    try:
        instructions_list = ast.literal_eval(instructions)
        cleaned_instructions = [item['display_text']for item in instructions_list]
        return cleaned_instructions
    except:
        return None


df['cleaned_instructions'] = df['instructions'].apply(clean_instructions)
df.to_csv('cleaned_food.csv', index=False)

In [25]:
df1 = pd.read_csv('cleaned_food.csv')
df1.head()

,name,video_url,thumbnail_url,ratings_positive,score,protein,fat,calories,sugar,carbohydrates,fiber,ingredient_sections,instructions,cleaned_ingredients,cleaned_instructions
0,Homemade Cinnamon Rolls,https://vid.tasty.co/output/25812/mp4_640x640/...,https://img.buzzfeed.com/video-api-prod/assets...,17340,98.0,7.0,21.0,479.0,24.0,63.0,1.0,"[{'name': 'Dough', 'ingredients': [{'extra_com...",[{'display_text': 'Generously butter two dispo...,"['Dough', 'unsalted butter', 'whole milk', 'gr...",['Generously butter two disposable foil pie/ca...
1,Whipped Coffee,https://vid.tasty.co/output/163616/landscape_4...,https://img.buzzfeed.com/video-api-prod/assets...,8483,92.0,0.0,0.0,69.0,18.0,18.0,0.0,"[{'name': '', 'ingredients': [{'extra_comment'...","[{'display_text': 'Add the hot water, sugar, a...","['hot water', 'sugar', 'instant coffee powder'...","['Add the hot water, sugar, and instant coffee..."
2,Fluffy Pancakes,https://vid.tasty.co/output/65088/landscape_48...,https://img.buzzfeed.com/thumbnailer-prod-us-e...,5009,83.0,36.0,50.0,1102.0,12.0,123.0,3.0,"[{'name': '', 'ingredients': [{'extra_comment'...",[{'display_text': 'Whisk together the flour an...,"['flour', 'baking powder', 'milk', 'butter', '...",['Whisk together the flour and baking powder i...
3,How to Make Cinnamon Rolls,https://vid.tasty.co/output/121022/landscape_4...,https://img.buzzfeed.com/thumbnailer-prod-us-e...,6649,97.0,8.0,25.0,562.0,28.0,74.0,1.0,"[{'name': 'Dough', 'ingredients': [{'extra_com...",[{'display_text': 'Make the dough: In a large ...,"['Dough', 'whole milk', 'sugar', 'unsalted but...","['Make the dough: In a large bowl, whisk toget..."
4,Healthy Banana Pancakes,https://vid.tasty.co/output/27116/mp4_640x640/...,https://img.buzzfeed.com/thumbnailer-prod-us-e...,3853,87.0,7.0,4.0,184.0,9.0,30.0,4.0,"[{'name': '', 'ingredients': [{'extra_comment'...",[{'display_text': 'Mash bananas in a large bow...,"['ripe bananas', 'eggs', 'vanilla extract', 'q...",['Mash bananas in a large bowl until smooth. M...


In [26]:
df1['Score'] = df1[['ratings_positive', 'score']].sum(axis=1)
df1.head()

,name,video_url,thumbnail_url,ratings_positive,score,protein,fat,calories,sugar,carbohydrates,fiber,ingredient_sections,instructions,cleaned_ingredients,cleaned_instructions,Score
0,Homemade Cinnamon Rolls,https://vid.tasty.co/output/25812/mp4_640x640/...,https://img.buzzfeed.com/video-api-prod/assets...,17340,98.0,7.0,21.0,479.0,24.0,63.0,1.0,"[{'name': 'Dough', 'ingredients': [{'extra_com...",[{'display_text': 'Generously butter two dispo...,"['Dough', 'unsalted butter', 'whole milk', 'gr...",['Generously butter two disposable foil pie/ca...,17438.0
1,Whipped Coffee,https://vid.tasty.co/output/163616/landscape_4...,https://img.buzzfeed.com/video-api-prod/assets...,8483,92.0,0.0,0.0,69.0,18.0,18.0,0.0,"[{'name': '', 'ingredients': [{'extra_comment'...","[{'display_text': 'Add the hot water, sugar, a...","['hot water', 'sugar', 'instant coffee powder'...","['Add the hot water, sugar, and instant coffee...",8575.0
2,Fluffy Pancakes,https://vid.tasty.co/output/65088/landscape_48...,https://img.buzzfeed.com/thumbnailer-prod-us-e...,5009,83.0,36.0,50.0,1102.0,12.0,123.0,3.0,"[{'name': '', 'ingredients': [{'extra_comment'...",[{'display_text': 'Whisk together the flour an...,"['flour', 'baking powder', 'milk', 'butter', '...",['Whisk together the flour and baking powder i...,5092.0
3,How to Make Cinnamon Rolls,https://vid.tasty.co/output/121022/landscape_4...,https://img.buzzfeed.com/thumbnailer-prod-us-e...,6649,97.0,8.0,25.0,562.0,28.0,74.0,1.0,"[{'name': 'Dough', 'ingredients': [{'extra_com...",[{'display_text': 'Make the dough: In a large ...,"['Dough', 'whole milk', 'sugar', 'unsalted but...","['Make the dough: In a large bowl, whisk toget...",6746.0
4,Healthy Banana Pancakes,https://vid.tasty.co/output/27116/mp4_640x640/...,https://img.buzzfeed.com/thumbnailer-prod-us-e...,3853,87.0,7.0,4.0,184.0,9.0,30.0,4.0,"[{'name': '', 'ingredients': [{'extra_comment'...",[{'display_text': 'Mash bananas in a large bow...,"['ripe bananas', 'eggs', 'vanilla extract', 'q...",['Mash bananas in a large bowl until smooth. M...,3940.0


In [27]:
df1 = df1.drop(['score', 'ratings_positive', 'instructions'], axis=1)
df1.head()

,name,video_url,thumbnail_url,protein,fat,calories,sugar,carbohydrates,fiber,ingredient_sections,cleaned_ingredients,cleaned_instructions,Score
0,Homemade Cinnamon Rolls,https://vid.tasty.co/output/25812/mp4_640x640/...,https://img.buzzfeed.com/video-api-prod/assets...,7.0,21.0,479.0,24.0,63.0,1.0,"[{'name': 'Dough', 'ingredients': [{'extra_com...","['Dough', 'unsalted butter', 'whole milk', 'gr...",['Generously butter two disposable foil pie/ca...,17438.0
1,Whipped Coffee,https://vid.tasty.co/output/163616/landscape_4...,https://img.buzzfeed.com/video-api-prod/assets...,0.0,0.0,69.0,18.0,18.0,0.0,"[{'name': '', 'ingredients': [{'extra_comment'...","['hot water', 'sugar', 'instant coffee powder'...","['Add the hot water, sugar, and instant coffee...",8575.0
2,Fluffy Pancakes,https://vid.tasty.co/output/65088/landscape_48...,https://img.buzzfeed.com/thumbnailer-prod-us-e...,36.0,50.0,1102.0,12.0,123.0,3.0,"[{'name': '', 'ingredients': [{'extra_comment'...","['flour', 'baking powder', 'milk', 'butter', '...",['Whisk together the flour and baking powder i...,5092.0
3,How to Make Cinnamon Rolls,https://vid.tasty.co/output/121022/landscape_4...,https://img.buzzfeed.com/thumbnailer-prod-us-e...,8.0,25.0,562.0,28.0,74.0,1.0,"[{'name': 'Dough', 'ingredients': [{'extra_com...","['Dough', 'whole milk', 'sugar', 'unsalted but...","['Make the dough: In a large bowl, whisk toget...",6746.0
4,Healthy Banana Pancakes,https://vid.tasty.co/output/27116/mp4_640x640/...,https://img.buzzfeed.com/thumbnailer-prod-us-e...,7.0,4.0,184.0,9.0,30.0,4.0,"[{'name': '', 'ingredients': [{'extra_comment'...","['ripe bananas', 'eggs', 'vanilla extract', 'q...",['Mash bananas in a large bowl until smooth. M...,3940.0


In [28]:
df1.to_csv('Food_Dataset.csv', index=False)

In [29]:
df1.head()

,name,video_url,thumbnail_url,protein,fat,calories,sugar,carbohydrates,fiber,ingredient_sections,cleaned_ingredients,cleaned_instructions,Score
0,Homemade Cinnamon Rolls,https://vid.tasty.co/output/25812/mp4_640x640/...,https://img.buzzfeed.com/video-api-prod/assets...,7.0,21.0,479.0,24.0,63.0,1.0,"[{'name': 'Dough', 'ingredients': [{'extra_com...","['Dough', 'unsalted butter', 'whole milk', 'gr...",['Generously butter two disposable foil pie/ca...,17438.0
1,Whipped Coffee,https://vid.tasty.co/output/163616/landscape_4...,https://img.buzzfeed.com/video-api-prod/assets...,0.0,0.0,69.0,18.0,18.0,0.0,"[{'name': '', 'ingredients': [{'extra_comment'...","['hot water', 'sugar', 'instant coffee powder'...","['Add the hot water, sugar, and instant coffee...",8575.0
2,Fluffy Pancakes,https://vid.tasty.co/output/65088/landscape_48...,https://img.buzzfeed.com/thumbnailer-prod-us-e...,36.0,50.0,1102.0,12.0,123.0,3.0,"[{'name': '', 'ingredients': [{'extra_comment'...","['flour', 'baking powder', 'milk', 'butter', '...",['Whisk together the flour and baking powder i...,5092.0
3,How to Make Cinnamon Rolls,https://vid.tasty.co/output/121022/landscape_4...,https://img.buzzfeed.com/thumbnailer-prod-us-e...,8.0,25.0,562.0,28.0,74.0,1.0,"[{'name': 'Dough', 'ingredients': [{'extra_com...","['Dough', 'whole milk', 'sugar', 'unsalted but...","['Make the dough: In a large bowl, whisk toget...",6746.0
4,Healthy Banana Pancakes,https://vid.tasty.co/output/27116/mp4_640x640/...,https://img.buzzfeed.com/thumbnailer-prod-us-e...,7.0,4.0,184.0,9.0,30.0,4.0,"[{'name': '', 'ingredients': [{'extra_comment'...","['ripe bananas', 'eggs', 'vanilla extract', 'q...",['Mash bananas in a large bowl until smooth. M...,3940.0


In [30]:
df1 = pd.read_csv('Food_Dataset.csv')

<h1 style="font-family: 'Times New Roman'; letter-spacing: 0.08em;">Making Demo Recommendation</h1>

In [31]:
df1['cleaned_ingredients'] = df1['cleaned_ingredients'].apply(lambda x: x.replace('[', '').replace(']', '').replace("'", '').split(', '))

vectorizer = TfidfVectorizer()
tfid_vectorizer = vectorizer.fit_transform(df1['cleaned_ingredients'].apply(lambda x: ', '.join(x)))

In [32]:
user_input = ['chicken', 'cheese']  # Replace with any ingredeints 
user_input_vector = vectorizer.transform([', '.join(user_input)])

cosine_similarities = cosine_similarity(tfid_vectorizer, user_input_vector).flatten()

top_5_indices = cosine_similarities.argsort()[-5:][::-1]

recommended_recipes = df1.iloc[top_5_indices]

recommended_titles = recommended_recipes['name']
print(recommended_titles)

587     Cheesy Chicken Alfredo Pasta Bake
991         Butter Chicken Mac ‘n’ Cheese
621                    Hasselback Chicken
694          Cheesy Chicken Broccoli Bake
1613              Cheesy Chicken Pot Pies
Name: name, dtype: object


## Notes

1 - you may see some additional lines Which seems not necessary but if i didnt make it , some errors will appear out of no where

2 - i keept the cplumn ' score' so if there is need to ordering the recommendation not only based on similarity but also on its score 

3 - choosing these dataset even if it seems kind small because we need images and video to be displayed so i didnt found anything else

here is dataset link :  https://www.kaggle.com/datasets/zeeenb/recipes-from-tasty